### ERA5 Data Processing Script

This Python script processes **raw hourly ERA5 GRIB files** into clean, model-ready datasets for a specific location (Minneapolis).

##### Key Objectives

- **Custom Year Selection**  
  - Training: **2000–2020**  
  - Testing: **2021–2024**  
  - Enforces strict separation to prevent data leakage.

- **Location-Specific Extraction**  
  - Identifies and extracts data from the **nearest ERA5 grid point** to Minneapolis.

- **Variable Harmonization & Unit Conversion**  
  - Converts U/V wind components → **scalar wind speed**  
  - Air temp & dewpoint → **relative humidity**  
  - Cumulative variables (e.g. precip, radiation) → **hourly totals/rates**

##### Workflow

1. Loop through each year in training & testing periods  
2. Read GRIB files, extract hourly series for Minneapolis  
3. Apply conversions and standardize variables  
4. Concatenate results

##### Final Outputs

- `ERA5_Train_2000-2020.csv` – Clean hourly data for training  
- `ERA5_Test_2021-2024.csv` – Clean hourly data for testing

##### Note: 
- Due to processing reasons, I have to unzip the files manually rather than automating it.

In [2]:
import os
import zipfile
import pygrib
import pandas as pd
import numpy as np

# --- 1. CONFIGURATION ---
# This section defines the data sources and the train/test split.

ROOT_DATA_DIR = r"C:\Users\91788\Downloads\ERA5 Data\Extracted"

# Define the years for the training set and the test set
TRAIN_YEARS = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
TEST_YEARS = [2021,2022,2023,2024]

MINNEAPOLIS_COORDS = {"lat": 44.98, "lon": -93.26}

# This mapping ensures we use consistent names throughout the process.
# Using the new, better variable names from your latest download.
VARIABLE_MAPPING = {
    '2 metre temperature': '2_metre_temperature',
    '10 metre U wind component': '10_metre_u_wind_component',
    '10 metre V wind component': '10_metre_v_wind_component',
    '2 metre dewpoint temperature': '2_metre_dewpoint_temperature',
    'Total precipitation': 'total_precipitation',
    'Mean surface downward short-wave radiation flux': 'solar_radiation',
    'Mean surface downward long-wave radiation flux': 'thermal_radiation'
}
DESIRED_GRIB_VARIABLES = list(VARIABLE_MAPPING.keys())

# --- 2. HELPER & UNIT CONVERSION FUNCTIONS ---

def decumulate_hourly(series: pd.Series) -> pd.Series:
    """
    Robustly de-cumulates an hourly time series that may have forecast resets.
    """
    diffs = series.diff()
    # Where diff is negative (a reset), use the original series value.
    hourly_values = diffs.where(diffs >= 0, series)
    # The first value will be NaN after diff(), so fill it with the original first value.
    if len(hourly_values) > 0:
        hourly_values.iloc[0] = series.iloc[0]
    return hourly_values

def standardize_dataframe(raw_df):
    """
    Takes a raw DataFrame from GRIB extraction and converts all variables
    to their final, standardized units.
    """
    print("  Applying unit conversions...")
    df = raw_df.copy()

    df['wind_speed_ms'] = np.sqrt(df[VARIABLE_MAPPING['10 metre U wind component']]**2 + df[VARIABLE_MAPPING['10 metre V wind component']]**2)

    precip_m = decumulate_hourly(df[VARIABLE_MAPPING['Total precipitation']])
    df['precip_hourly_mm'] = precip_m * 1000

    t_air_c = df[VARIABLE_MAPPING['2 metre temperature']] - 273.15
    t_dew_c = df[VARIABLE_MAPPING['2 metre dewpoint temperature']] - 273.15
    e_s = 0.61094 * np.exp((17.625 * t_air_c) / (t_air_c + 243.04))
    e = 0.61094 * np.exp((17.625 * t_dew_c) / (t_dew_c + 243.04))
    df['relative_humidity_percent'] = (e / e_s) * 100
    df['relative_humidity_percent'] = df['relative_humidity_percent'].clip(0, 100)

    solar_rad_j_m2 = decumulate_hourly(df[VARIABLE_MAPPING['Mean surface downward short-wave radiation flux']])
    df['solar_radiation_w_m2'] = solar_rad_j_m2 / 3600

    thermal_rad_j_m2 = decumulate_hourly(df[VARIABLE_MAPPING['Mean surface downward long-wave radiation flux']])
    df['thermal_radiation_w_m2'] = thermal_rad_j_m2 / 3600

    df = df.rename(columns={VARIABLE_MAPPING['2 metre temperature']: 'air_temperature_k'})
    final_cols = [
        'air_temperature_k', 'wind_speed_ms', 'precip_hourly_mm',
        'relative_humidity_percent', 'solar_radiation_w_m2', 'thermal_radiation_w_m2'
    ]
    return df[final_cols]

# --- 3. MAIN PROCESSING WORKFLOW ---

def extract_raw_data_for_year(grib_file_path, target_coords):
    """Extracts a raw time series DataFrame for a single point from a GRIB file."""
    time_series_data = {}
    with pygrib.open(grib_file_path) as grbs:
        msgs = list(grbs)
    if not msgs: return None
    
    grb_grid_msg = msgs[0]
    lats, lons = grb_grid_msg.latlons()
    lons_180 = (lons + 180) % 360 - 180
    dist_sq = (lats - target_coords["lat"])**2 + (lons_180 - target_coords["lon"])**2
    min_dist_idx = np.unravel_index(np.argmin(dist_sq, axis=None), dist_sq.shape)
    print(f"  Found nearest grid point for {target_coords} at index {min_dist_idx}.")
    
    for grb in msgs:
        if grb.name in DESIRED_GRIB_VARIABLES:
            ts = grb.validDate
            clean_name = VARIABLE_MAPPING[grb.name]
            if ts not in time_series_data: time_series_data[ts] = {'time': ts}
            time_series_data[ts][clean_name] = grb.values[min_dist_idx]
            
    if not time_series_data: return None
    df = pd.DataFrame(sorted(time_series_data.values(), key=lambda x: x['time'])).set_index('time')
    return df

def process_era5_data_for_years(base_dir, years_list):
    """
    Main function to find, extract, and process ERA5 GRIB files for a given list of years.
    This version assumes files have already been unzipped.
    """
    all_yearly_data = []
    for year in sorted(years_list):
        year_str = str(year)
        print(f"\n--- Processing Year: {year_str} ---")

        # The script now looks for a folder with the year's name directly.
        year_folder = os.path.join(base_dir, year_str)
        
        if not os.path.isdir(year_folder):
            print(f"  Error: Directory not found at '{year_folder}'. Skipping year.")
            continue

        # Find the GRIB file within the year's folder
        grib_file_path = None
        for file in os.listdir(year_folder):
            if file.lower().endswith(('.grib', '.grb')):
                grib_file_path = os.path.join(year_folder, file)
                break
        
        if not grib_file_path:
            print(f"  Error: No GRIB file found in '{year_folder}'. Skipping year.")
            continue
        
        print(f"  Loading raw data from: {os.path.basename(grib_file_path)}")
        raw_df = extract_raw_data_for_year(grib_file_path, MINNEAPOLIS_COORDS)
        
        if raw_df is None:
            print(f"  Could not extract raw data for {year_str}. Skipping.")
            continue
            
        standardized_df = standardize_dataframe(raw_df)
        all_yearly_data.append(standardized_df)
        print(f"  Successfully processed and standardized data for {year_str}.")

    if not all_yearly_data:
        print("\nNo data was successfully processed.")
        return None

    print("\nCombining all processed years into a single DataFrame...")
    master_df = pd.concat(all_yearly_data)
    return master_df

# --- 4. EXECUTION ---
if __name__ == "__main__":
    
    # --- Process Training Data ---
    print("\n\n=============== PROCESSING TRAINING DATA ===============")
    train_df = process_era5_data_for_years(ROOT_DATA_DIR, TRAIN_YEARS)
    
    if train_df is not None:
        print("\n--- Final Training DataFrame ---")
        print(train_df.head())
        print(f"\nTraining data shape: {train_df.shape}")
        
        output_path_train = os.path.join(ROOT_DATA_DIR, "ERA5_Train_2000-2020.csv")
        print(f"\nSaving training data to: {output_path_train}")
        train_df.to_csv(output_path_train)
        print("Save complete.")

    # --- Process Testing Data ---
    print("\n\n=============== PROCESSING TESTING DATA ===============")
    test_df = process_era5_data_for_years(ROOT_DATA_DIR, TEST_YEARS)

    if test_df is not None:
        print("\n--- Final Testing DataFrame ---")
        print(test_df.head())
        print(f"\nTesting data shape: {test_df.shape}")

        output_path_test = os.path.join(ROOT_DATA_DIR, "ERA5_Test_2021-2024.csv")
        print(f"\nSaving testing data to: {output_path_test}")
        test_df.to_csv(output_path_test)
        print("Save complete.")




=============== PROCESSING TRAINING DATA ===============

--- Processing Year: 2000 ---
  Loading raw data from: data.grib
  Found nearest grid point for {'lat': 44.98, 'lon': -93.26} at index (4, 3).
  Applying unit conversions...
  Successfully processed and standardized data for 2000.

--- Processing Year: 2001 ---
  Loading raw data from: data.grib
  Found nearest grid point for {'lat': 44.98, 'lon': -93.26} at index (4, 3).
  Applying unit conversions...
  Successfully processed and standardized data for 2001.

--- Processing Year: 2002 ---
  Loading raw data from: data.grib
  Found nearest grid point for {'lat': 44.98, 'lon': -93.26} at index (4, 3).
  Applying unit conversions...
  Successfully processed and standardized data for 2002.

--- Processing Year: 2003 ---
  Loading raw data from: data.grib
  Found nearest grid point for {'lat': 44.98, 'lon': -93.26} at index (4, 3).
  Applying unit conversions...
  Successfully processed and standardized data for 2003.

--- Processi

In [6]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURATION ---
# Define the location of your processed data files.
ROOT_DATA_DIR = r"C:\Users\91788\Downloads\ERA5 Data\Extracted"
TRAIN_INPUT_FILE = os.path.join(ROOT_DATA_DIR, "ERA5_Train_2000-2020.csv")
TEST_INPUT_FILE = os.path.join(ROOT_DATA_DIR, "ERA5_Test_2021-2024.csv")

# Define where to save the final, model-ready data
TRAIN_OUTPUT_FILE = os.path.join(ROOT_DATA_DIR, "MODELING_Train_2000-2020.csv")
TEST_OUTPUT_FILE = os.path.join(ROOT_DATA_DIR, "MODELING_Test_2021-2024.csv")


# --- 2. DATA STRUCTURING FUNCTION ---

def create_modeling_datasets(hourly_df: pd.DataFrame):
    """
    Takes a clean hourly time series and structures it into two key datasets:
    1. A daily summary DataFrame to be used as model predictors (X).
    2. A pivoted hourly DataFrame to be used as model targets (y).
    """
    print("  Creating daily summary predictors (X)...")
    # --- Create Daily Predictors (X) ---
    # Resample the hourly data to a daily frequency ('D').
    # For each variable, calculate the relevant summary statistics.
    daily_predictors = hourly_df.resample('D').agg(
        # For temperature, we want the mean, min, and max for the day.
        air_temperature_k_mean=('air_temperature_k', 'mean'),
        air_temperature_k_min=('air_temperature_k', 'min'),
        air_temperature_k_max=('air_temperature_k', 'max'),
        # For wind, we want mean, max, and standard deviation (gustiness).
        wind_speed_ms_mean=('wind_speed_ms', 'mean'),
        wind_speed_ms_max=('wind_speed_ms', 'max'),
        wind_speed_ms_std=('wind_speed_ms', 'std'),
        # For other variables, the daily mean is sufficient.
        relative_humidity_percent_mean=('relative_humidity_percent', 'mean'),
        solar_radiation_w_m2_mean=('solar_radiation_w_m2', 'mean'),
        thermal_radiation_w_m2_mean=('thermal_radiation_w_m2', 'mean'),
        # For precipitation, we want the total sum for the day.
        precip_hourly_mm_sum=('precip_hourly_mm', 'sum')
    ).dropna() # Drop any days that might have missing data


    print("  Creating pivoted hourly targets (y)...")
    # --- Create Hourly Targets (y) ---
    # To create our target columns (e.g., temp_hour_0, temp_hour_1...),
    # we first need to add 'date' and 'hour' columns to the original data.
    df_for_pivot = hourly_df.copy()
    df_for_pivot['date'] = df_for_pivot.index.date
    df_for_pivot['hour'] = df_for_pivot.index.hour
    
    # Pivot the table to create one row per day and 24 columns for each hour
    hourly_targets = pd.pivot_table(
        df_for_pivot,
        index='date',
        columns='hour',
        # List all the variables you want to have hourly columns for
        values=['air_temperature_k', 'wind_speed_ms', 'precip_hourly_mm',
                'relative_humidity_percent', 'solar_radiation_w_m2', 'thermal_radiation_w_m2']
    )
    # The pivot creates multi-level columns; flatten them into a single level.
    hourly_targets.columns = [f"{var}_{hour}" for var, hour in hourly_targets.columns]
    
    
    print("  Joining predictors and targets...")
    # --- Combine into Final Modeling DataFrame ---
    # Merge the daily predictors and hourly targets on the date index.
    # This ensures perfect alignment between our X and y data.
    modeling_df = daily_predictors.join(hourly_targets, how='inner')
    
    return modeling_df

# --- 3. EXECUTION ---
if __name__ == "__main__":

    # --- Process Training Data ---
    print("\n=============== PREPARING TRAINING DATA ===============")
    try:
        train_hourly_df = pd.read_csv(TRAIN_INPUT_FILE, index_col='time', parse_dates=True)
        train_modeling_df = create_modeling_datasets(train_hourly_df)
        
        print("\n--- Final Training Data for Modeling ---")
        print(train_modeling_df.head())
        print(f"\nTraining modeling data shape: {train_modeling_df.shape}")
        
        print(f"\nSaving model-ready training data to: {TRAIN_OUTPUT_FILE}")
        train_modeling_df.to_csv(TRAIN_OUTPUT_FILE)
        print("Save complete.")
        
    except FileNotFoundError:
        print(f"Error: Training file not found at {TRAIN_INPUT_FILE}")

    # --- Process Testing Data ---
    print("\n\n=============== PREPARING TESTING DATA ===============")
    try:
        test_hourly_df = pd.read_csv(TEST_INPUT_FILE, index_col='time', parse_dates=True)
        test_modeling_df = create_modeling_datasets(test_hourly_df)
        
        print("\n--- Final Testing Data for Modeling ---")
        print(test_modeling_df.head())
        print(f"\nTesting modeling data shape: {test_modeling_df.shape}")

        print(f"\nSaving model-ready testing data to: {TEST_OUTPUT_FILE}")
        test_modeling_df.to_csv(TEST_OUTPUT_FILE)
        print("Save complete.")

    except FileNotFoundError:
        print(f"Error: Testing file not found at {TEST_INPUT_FILE}")


=============== PREPARING TRAINING DATA ===============
  Creating daily summary predictors (X)...
  Creating pivoted hourly targets (y)...
  Joining predictors and targets...

--- Final Training Data for Modeling ---
            air_temperature_k_mean  air_temperature_k_min  \
2000-01-01              271.268669             268.519287   
2000-01-02              271.413062             269.948181   
2000-01-03              269.083822             267.373535   
2000-01-04              265.343854             261.433716   
2000-01-05              263.126009             258.798325   

            air_temperature_k_max  wind_speed_ms_mean  wind_speed_ms_max  \
2000-01-01             275.477783            3.805613           5.798653   
2000-01-02             274.148315            4.017749           5.433529   
2000-01-03             271.614990            2.997503           3.560448   
2000-01-04             269.044189            4.446761           5.871527   
2000-01-05             268.758179 

In [10]:
# import pandas as pd
# import numpy as np
# import os
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# import joblib # Used for saving and loading models

# # --- 1. CONFIGURATION ---
# # Define the location of your modeling-ready data files.
# ROOT_DATA_DIR = r"C:\Users\91788\Downloads\ERA5 Data\Extracted"
# TRAIN_MODELING_FILE = os.path.join(ROOT_DATA_DIR, "MODELING_Train_2000-2014.csv")
# TEST_MODELING_FILE = os.path.join(ROOT_DATA_DIR, "MODELING_Test_2015-18.csv")
# MODEL_SAVE_DIR = os.path.join(ROOT_DATA_DIR, "trained_models")
# PREDICTIONS_OUTPUT_FILE = os.path.join(ROOT_DATA_DIR, "VALIDATION_Predictions_2015-18.csv")

# # Create a directory to save our trained models
# os.makedirs(MODEL_SAVE_DIR, exist_ok=True)


# # --- 2. LOAD PREPARED DATA ---
# print("--- Step 1: Loading prepared modeling data ---")
# try:
#     train_df = pd.read_csv(TRAIN_MODELING_FILE, index_col=0, parse_dates=True)
#     test_df = pd.read_csv(TEST_MODELING_FILE, index_col=0, parse_dates=True)
#     print("  Successfully loaded training and testing data.")
# except FileNotFoundError as e:
#     print(f"Error: Could not find data file at {e.filename}.")
#     print("Please ensure you have run the previous data preparation script successfully.")
#     exit() # Exit if data isn't found


# # --- 3. TRAIN THE MODELS ---
# print("\n--- Step 2: Training all downscaling models ---")

# # A dictionary to hold all our trained models
# trained_models = {}

# # --- 3a. Train "Daily Characteristics" Models for Wind (Two-Stage Approach) ---
# print("  Training Stage-1 wind characteristic models...")
# if 'wind_speed_ms_mean' in train_df.columns and 'wind_speed_ms_max' in train_df.columns and 'wind_speed_ms_std' in train_df.columns:
#     wind_predictors_train = train_df[['wind_speed_ms_mean']]
#     wind_target_max_train = train_df['wind_speed_ms_max']
#     wind_target_std_train = train_df['wind_speed_ms_std']

#     model_wind_max = LinearRegression()
#     model_wind_max.fit(wind_predictors_train, wind_target_max_train)
#     trained_models['wind_max_from_mean'] = model_wind_max
#     joblib.dump(model_wind_max, os.path.join(MODEL_SAVE_DIR, 'model_wind_max.pkl'))

#     model_wind_std = LinearRegression()
#     model_wind_std.fit(wind_predictors_train, wind_target_std_train)
#     trained_models['wind_std_from_mean'] = model_wind_std
#     joblib.dump(model_wind_std, os.path.join(MODEL_SAVE_DIR, 'model_wind_std.pkl'))
#     print("  Wind characteristic models trained and saved.")
# else:
#     print("  Warning: Could not train wind characteristic models due to missing columns.")


# # --- 3b. Train Main Hourly Models for Each Variable ---
# predictor_map = {
#     'air_temperature_k': ['air_temperature_k_mean', 'air_temperature_k_min', 'air_temperature_k_max'],
#     'wind_speed_ms': ['wind_speed_ms_mean', 'wind_speed_ms_max', 'wind_speed_ms_std'],
#     'relative_humidity_percent': ['relative_humidity_percent_mean'],
#     'solar_radiation_w_m2': ['solar_radiation_w_m2_mean'],
#     'thermal_radiation_w_m2': ['thermal_radiation_w_m2_mean'],
#     'precip_hourly_mm': ['precip_hourly_mm_sum']
# }

# for var_name, predictors in predictor_map.items():
#     print(f"  Training 24 hourly models for: {var_name}...")
#     trained_models[var_name] = {}
#     if not all(p in train_df.columns for p in predictors):
#         print(f"    Warning: Not all predictor columns for '{var_name}' found. Skipping.")
#         continue
#     for hour in range(24):
#         target_col = f"{var_name}_{hour}"
#         if target_col not in train_df.columns:
#             continue
#         X_train = train_df[predictors]
#         y_train = train_df[target_col]
#         model = LinearRegression()
#         model.fit(X_train, y_train)
#         trained_models[var_name][hour] = model
#     joblib.dump(trained_models[var_name], os.path.join(MODEL_SAVE_DIR, f'models_{var_name}.pkl'))

# print("--- All models have been trained successfully. ---")


# # --- 3.5. NEW: EVALUATE MODEL FIT ON TRAINING DATA (2005-2007) ---
# print("\n--- Step 2.5: Evaluating model fit on the training data ---")

# train_predictions = {}
# for var_name, predictors in predictor_map.items():
#     if var_name not in trained_models or not trained_models[var_name]: continue
    
#     hourly_preds_list = []
#     X_train_fit = train_df[predictors]
#     for hour in range(24):
#         model = trained_models[var_name].get(hour)
#         if model:
#             preds = model.predict(X_train_fit)
#             hourly_preds_list.append(pd.Series(preds, index=X_train_fit.index, name=hour))
    
#     if hourly_preds_list:
#         var_df_wide = pd.concat(hourly_preds_list, axis=1)
#         var_stacked = var_df_wide.stack()
#         var_stacked.index = var_stacked.index.map(lambda x: x[0] + pd.to_timedelta(x[1], unit='h'))
#         train_predictions[f'predicted_{var_name}'] = var_stacked

# train_predictions_df = pd.DataFrame(train_predictions)

# # Prepare actuals from training data for comparison
# train_actuals_df = pd.DataFrame()
# for var_name in predictor_map.keys():
#     actual_cols = [f"{var_name}_{h}" for h in range(24) if f"{var_name}_{h}" in train_df.columns]
#     if not actual_cols: continue
#     actual_hourly = train_df[actual_cols]
#     actual_hourly.columns = [int(c.split('_')[-1]) for c in actual_cols]
#     actual_stacked = actual_hourly.stack()
#     actual_stacked.index = actual_stacked.index.map(lambda x: x[0] + pd.to_timedelta(x[1], unit='h'))
#     train_actuals_df[f'actual_{var_name}'] = actual_stacked

# train_validation_df = pd.merge(train_actuals_df, train_predictions_df, left_index=True, right_index=True, how="inner")

# print("  Training Set Validation Results (Goodness of Fit):")
# for var_name in predictor_map.keys():
#     actual_col = f'actual_{var_name}'
#     predicted_col = f'predicted_{var_name}'
#     if actual_col in train_validation_df.columns and predicted_col in train_validation_df.columns:
#         temp_compare_df = train_validation_df[[actual_col, predicted_col]].dropna()
#         if not temp_compare_df.empty:
#             mae = mean_absolute_error(temp_compare_df[actual_col], temp_compare_df[predicted_col])
#             rmse = np.sqrt(mean_squared_error(temp_compare_df[actual_col], temp_compare_df[predicted_col]))
#             print(f"    - {var_name}:")
#             print(f"        MAE:  {mae:.4f}")
#             print(f"        RMSE: {rmse:.4f}")

# # --- 4. MAKE PREDICTIONS ON THE TEST SET (2008) ---
# print("\n--- Step 3: Generating hourly predictions for the test year (2008) ---")

# X_test_base = test_df.copy()

# if 'wind_max_from_mean' in trained_models and 'wind_std_from_mean' in trained_models:
#     print("  Generating wind characteristic predictions for test set...")
#     X_test_wind_mean = test_df[['wind_speed_ms_mean']]
#     predicted_wind_max = trained_models['wind_max_from_mean'].predict(X_test_wind_mean)
#     predicted_wind_std = trained_models['wind_std_from_mean'].predict(X_test_wind_mean)
#     X_test_base['wind_speed_ms_max'] = predicted_wind_max
#     X_test_base['wind_speed_ms_std'] = predicted_wind_std
#     print("  Wind characteristics generated.")

# final_predictions = {}
# for var_name, predictors in predictor_map.items():
#     if var_name not in trained_models or not trained_models[var_name]:
#         print(f"  Skipping prediction for '{var_name}' as no models were trained.")
#         continue
#     if not all(p in X_test_base.columns for p in predictors):
#         print(f"    Warning: Not all predictor columns for '{var_name}' found in test set. Skipping prediction.")
#         continue
#     print(f"  Predicting hourly values for: {var_name}...")
#     hourly_preds_list = []
#     for hour in range(24):
#         model = trained_models[var_name].get(hour)
#         if model:
#             X_test_var = X_test_base[predictors]
#             preds = model.predict(X_test_var)
#             hourly_preds_list.append(pd.Series(preds, index=X_test_base.index, name=hour))
#     if hourly_preds_list:
#         var_df_wide = pd.concat(hourly_preds_list, axis=1)
#         var_stacked = var_df_wide.stack()
#         var_stacked.index = var_stacked.index.map(lambda x: x[0] + pd.to_timedelta(x[1], unit='h'))
#         final_predictions[f'predicted_{var_name}'] = var_stacked
# predictions_df = pd.DataFrame(final_predictions)
# print("--- Hourly predictions generated successfully. ---")


# # --- 5. VALIDATE PREDICTIONS ON TEST DATA ---
# if not predictions_df.empty:
#     print("\n--- Step 4: Validating predictions against actual 2008 data (Test Set) ---")
    
#     actuals_df = pd.DataFrame()
#     for var_name in predictor_map.keys():
#         actual_cols = [f"{var_name}_{h}" for h in range(24) if f"{var_name}_{h}" in test_df.columns]
#         if not actual_cols: continue
#         actual_hourly = test_df[actual_cols]
#         actual_hourly.columns = [int(c.split('_')[-1]) for c in actual_cols]
#         actual_stacked = actual_hourly.stack()
#         actual_stacked.index = actual_stacked.index.map(lambda x: x[0] + pd.to_timedelta(x[1], unit='h'))
#         actuals_df[f'actual_{var_name}'] = actual_stacked
        
#     validation_df = pd.merge(actuals_df, predictions_df, left_index=True, right_index=True, how="inner")
    
#     print("  Test Set Validation Results (Generalization Performance):")
#     for var_name in predictor_map.keys():
#         actual_col = f'actual_{var_name}'
#         predicted_col = f'predicted_{var_name}'
#         if actual_col in validation_df.columns and predicted_col in validation_df.columns:
#             temp_compare_df = validation_df[[actual_col, predicted_col]].dropna()
#             if not temp_compare_df.empty:
#                 mae = mean_absolute_error(temp_compare_df[actual_col], temp_compare_df[predicted_col])
#                 rmse = np.sqrt(mean_squared_error(temp_compare_df[actual_col], temp_compare_df[predicted_col]))
#                 print(f"    - {var_name}:")
#                 print(f"        MAE:  {mae:.4f}")
#                 print(f"        RMSE: {rmse:.4f}")
#             else:
#                 print(f"    - {var_name}: Could not be calculated.")

#     # --- 6. SAVE FINAL PREDICTIONS ---
#     print(f"\n--- Step 5: Saving final 2008 predictions to {PREDICTIONS_OUTPUT_FILE} ---")
#     validation_df.to_csv(PREDICTIONS_OUTPUT_FILE)
#     print("Save complete.")
# else:
#     print("\nSkipping validation and saving because no predictions were generated.")


In [12]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import os

# # --- 1. CONFIGURATION ---
# # Define the location of your validation file.
# ROOT_DATA_DIR = r"C:\Users\91788\Downloads\ERA5 Data\Extracted"
# VALIDATION_FILE = os.path.join(ROOT_DATA_DIR, "VALIDATION_Predictions_2015-18.csv")
# PLOT_SAVE_DIR = os.path.join(ROOT_DATA_DIR, "validation_plots_Jan2016_1-3")

# # Create a directory to save the plots
# os.makedirs(PLOT_SAVE_DIR, exist_ok=True)

# # --- 2. DATA LOADING ---
# print("--- Loading validation data ---")
# try:
#     # FIXED: Use index_col=0 to specify that the first column is the index,
#     # regardless of its header name.
#     validation_df = pd.read_csv(VALIDATION_FILE, index_col=0, parse_dates=True)
#     print("  Successfully loaded validation data.")
# except FileNotFoundError:
#     print(f"Error: Validation file not found at {VALIDATION_FILE}")
#     exit()

# # --- 3. PLOTTING FUNCTION ---

# def plot_validation_timeseries(df, var_name, start_date, end_date):
#     """
#     Creates and saves a time series plot comparing actual vs. predicted values
#     for a specific variable and date range.
#     """
#     actual_col = f'actual_{var_name}'
#     predicted_col = f'predicted_{var_name}'
    
#     # Check if both required columns exist in the DataFrame
#     if not all(col in df.columns for col in [actual_col, predicted_col]):
#         print(f"\nSkipping plot for '{var_name}': one or both columns not found.")
#         return

#     # Filter the DataFrame for the desired date range
#     plot_df = df.loc[start_date:end_date]
    
#     if plot_df.empty:
#         print(f"\nNo data found for '{var_name}' in the date range {start_date} to {end_date}.")
#         return

#     print(f"\nGenerating plot for '{var_name}' from {start_date} to {end_date}...")

#     # Create the plot
#     plt.style.use('seaborn-v0_8-whitegrid')
#     fig, ax = plt.subplots(figsize=(15, 7))
    
#     # Plot the actual and predicted lines
#     ax.plot(plot_df.index, plot_df[actual_col], label='Actual (ERA5)', color='blue', linewidth=2, marker='o', markersize=4)
#     ax.plot(plot_df.index, plot_df[predicted_col], label='Predicted (Model)', color='red', linewidth=1.5, linestyle='--')
    
#     # Formatting the plot
#     plt.title(f'Actual vs. Predicted Hourly {var_name.replace("_", " ").title()}\n({start_date} to {end_date})', fontsize=16)
#     plt.ylabel(var_name.split('_')[-1].upper(), fontsize=12)
#     plt.xlabel('Date and Time', fontsize=12)
#     plt.legend(fontsize=12)
#     plt.xticks(rotation=45)
#     plt.tight_layout()
    
#     # Save the plot to a file
#     plot_filename = f"Validation_Plot_{var_name}_{start_date}_to_{end_date}.png"
#     save_path = os.path.join(PLOT_SAVE_DIR, plot_filename)
#     plt.savefig(save_path)
#     print(f"  Plot saved to: {save_path}")
#     plt.close(fig) # Close the figure to free up memory


# # --- 4. EXECUTION ---
# if __name__ == "__main__":
    
#     # Define the date range for the plots (e.g., the first 3 days of the year)
#     start_plot_date = '2016-01-01'
#     end_plot_date = '2016-01-03'

#     # Generate plots for all key variables
#     variables_to_plot = [
#         'air_temperature_k',
#         'wind_speed_ms',
#         'relative_humidity_percent',
#         'precip_hourly_mm',
#         'solar_radiation_w_m2',
#         'thermal_radiation_w_m2'
#     ]

#     for variable in variables_to_plot:
#         plot_validation_timeseries(validation_df, variable, start_plot_date, end_plot_date)
        
#     print("\n--- All plots generated successfully. ---")


In [9]:
# import pandas as pd
# import numpy as np
# import os
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# import joblib
# import matplotlib.pyplot as plt

# # --- 1. CONFIGURATION ---
# # Define the paths for your input files and trained models.
# ROOT_DATA_DIR = r"C:\Users\91788\Downloads\ERA5 Data\Extracted" 
# MODEL_SAVE_DIR = os.path.join(ROOT_DATA_DIR, "trained_models")

# # Input files for this final validation run
# NASA_DAILY_INPUT_FILE = os.path.join(ROOT_DATA_DIR, "NASA_Standardized_Minnesota_2015-2018.csv")
# ERA5_HOURLY_GROUND_TRUTH_FILE = os.path.join(ROOT_DATA_DIR, "ERA5_Standardized_Minneapolis_2015-2018.csv") 

# # Target coordinates for filtering the NASA data
# TARGET_LAT = 44.98
# TARGET_LON = -93.26

# # Output files
# FINAL_VALIDATION_OUTPUT_FILE = os.path.join(ROOT_DATA_DIR, "FINAL_VALIDATION_NASA_vs_ERA5_2015-2018.csv")
# PLOT_SAVE_DIR = os.path.join(ROOT_DATA_DIR, "final_validation_plots")
# os.makedirs(PLOT_SAVE_DIR, exist_ok=True)


# # --- 2. LOAD DATA AND TRAINED MODELS ---
# print("--- Step 1: Loading data and trained models for FINAL validation ---")

# # Load the multi-point, daily NASA data
# try:
#     nasa_df_full = pd.read_csv(NASA_DAILY_INPUT_FILE)
#     nasa_df_full['time'] = pd.to_datetime(nasa_df_full['time'])
#     print("  Successfully loaded full NASA daily data (2015-2018).")
# except FileNotFoundError:
#     print(f"Error: NASA data file not found at {NASA_DAILY_INPUT_FILE}.")
#     exit()

# # Load the single-point, hourly ERA5 data (our ground truth)
# try:
#     era5_df = pd.read_csv(ERA5_HOURLY_GROUND_TRUTH_FILE, index_col='time', parse_dates=True)
#     print("  Successfully loaded ERA5 hourly ground truth data (2015-2018).")
# except FileNotFoundError:
#     print(f"Error: ERA5 ground truth file not found at {ERA5_HOURLY_GROUND_TRUTH_FILE}.")
#     exit()

# # Load the library of trained models
# try:
#     print("  Loading pre-trained models...")
#     predictor_map = {
#         'air_temperature_k': ['tas', 'tasmin', 'tasmax'],
#         'wind_speed_ms': ['sfcWind_mean', 'sfcWind_max', 'sfcWind_std'],
#         'relative_humidity_percent': ['hurs'],
#         'solar_radiation_w_m2': ['rsds'],
#         'thermal_radiation_w_m2': ['rlds'],
#         'precip_hourly_mm': ['precip_daily_mm']
#     }
#     trained_models = {}
#     for var_name in predictor_map.keys():
#         model_path = os.path.join(MODEL_SAVE_DIR, f'models_{var_name}.pkl')
#         trained_models[var_name] = joblib.load(model_path)
    
#     trained_models['wind_max_from_mean'] = joblib.load(os.path.join(MODEL_SAVE_DIR, 'model_wind_max.pkl'))
#     trained_models['wind_std_from_mean'] = joblib.load(os.path.join(MODEL_SAVE_DIR, 'model_wind_std.pkl'))
#     print("  All models loaded successfully.")
# except FileNotFoundError as e:
#     print(f"Error loading model file: {e.filename}. Please ensure training was successful.")
#     exit()


# # --- 3. FILTER AND PREPARE NASA DATA FOR PREDICTION ---
# print("\n--- Step 2: Filtering and Preparing NASA data for Minneapolis ---")

# # Find the single closest point in the NASA data to our ERA5 point
# nasa_df_full['lon_180'] = (nasa_df_full['lon'] + 180) % 360 - 180
# unique_coords = nasa_df_full[['lat', 'lon', 'lon_180']].drop_duplicates()
# dist_sq = (unique_coords['lat'] - TARGET_LAT)**2 + (unique_coords['lon_180'] - TARGET_LON)**2
# closest_point = unique_coords.loc[dist_sq.idxmin()]

# print(f"  Filtering NASA data for closest point: Lat={closest_point['lat']}, Lon={closest_point['lon_180']:.2f}")
# nasa_daily_df = nasa_df_full[
#     (nasa_df_full['lat'] == closest_point['lat']) & 
#     (nasa_df_full['lon'] == closest_point['lon'])
# ].copy().set_index('time')

# # Create the full predictor set
# nasa_predictors_df = pd.DataFrame(index=nasa_daily_df.index)
# for var_name, predictors in predictor_map.items():
#     for p in predictors:
#         # Map NASA source columns to the names the model expects
#         source_col = p.split('_')[0] # e.g., 'tasmin' from 'air_temperature_k_min'
#         if source_col == 'sfcWind': source_col = 'sfcWind' # Special case for wind mean
#         if source_col == 'precip': source_col = 'pr' # Special case for precip sum

#         if source_col in nasa_daily_df.columns:
#             if source_col == 'pr':
#                  nasa_predictors_df[p] = nasa_daily_df[source_col] * 86400
#             else:
#                  nasa_predictors_df[p] = nasa_daily_df[source_col]

# # Use the Stage 1 models to predict max and std dev for wind
# X_wind_mean = nasa_predictors_df[['sfcWind_mean']]
# nasa_predictors_df['sfcWind_max'] = trained_models['wind_max_from_mean'].predict(X_wind_mean)
# nasa_predictors_df['sfcWind_std'] = trained_models['wind_std_from_mean'].predict(X_wind_mean)
# print("  NASA predictor data prepared.")

# # --- 4. GENERATE HOURLY PREDICTIONS ---
# print("\n--- Step 3: Generating hourly predictions from NASA daily data ---")
# final_predictions = {}
# for var_name, predictors in predictor_map.items():
#     if var_name not in trained_models or not trained_models[var_name] or not all(p in nasa_predictors_df.columns for p in predictors): continue
    
#     print(f"  Predicting hourly values for: {var_name}...")
#     hourly_preds_list = []
#     X_predict = nasa_predictors_df[predictors]
#     for hour in range(24):
#         model = trained_models[var_name].get(hour)
#         if model:
#             preds = model.predict(X_predict)
#             hourly_preds_list.append(pd.Series(preds, index=X_predict.index, name=hour))
#     if hourly_preds_list:
#         var_df_wide = pd.concat(hourly_preds_list, axis=1)
#         var_stacked = var_df_wide.stack()
#         var_stacked.index = var_stacked.index.map(lambda x: x[0] + pd.to_timedelta(x[1], unit='h'))
#         final_predictions[f'predicted_{var_name}'] = var_stacked

# predictions_df = pd.DataFrame(final_predictions)
# print("--- Hourly predictions generated successfully. ---")


# # --- 5. VALIDATE PREDICTIONS AND PLOT ---
# print("\n--- Step 4: Validating and visualizing results ---")

# validation_df = pd.merge(
#     era5_df.rename(columns=lambda c: f"actual_{c}"), 
#     predictions_df, 
#     left_index=True, 
#     right_index=True,
#     how="inner"
# )

# print("  Final Validation Results (NASA-based Predictions vs. ERA5 Actuals):")
# for var_name in predictor_map.keys():
#     actual_col, predicted_col = f'actual_{var_name}', f'predicted_{var_name}'
#     if actual_col in validation_df.columns and predicted_col in validation_df.columns:
#         temp_compare_df = validation_df[[actual_col, predicted_col]].dropna()
#         if not temp_compare_df.empty:
#             mae = mean_absolute_error(temp_compare_df[actual_col], temp_compare_df[predicted_col])
#             rmse = np.sqrt(mean_squared_error(temp_compare_df[actual_col], temp_compare_df[predicted_col]))
#             print(f"    - {var_name}:")
#             print(f"        MAE:  {mae:.4f}")
#             print(f"        RMSE: {rmse:.4f}")

# # --- Plotting ---
# start_plot_date = f'{min(SELECTED_YEARS)}-01-01'
# end_plot_date = f'{min(SELECTED_YEARS)}-01-03'

# for var_name in predictor_map.keys():
#     # Code to generate plots as in the previous visualization script
#     pass # Placeholder for brevity, plotting logic is the same

# # --- 6. SAVE FINAL RESULTS ---
# print(f"\n--- Step 5: Saving final validation results to {FINAL_VALIDATION_OUTPUT_FILE} ---")
# validation_df.to_csv(FINAL_VALIDATION_OUTPUT_FILE)
# print("Save complete.")

